我們需要更多決策避免過度擬和，也需要考慮更多的市場因素：

- 市場走向
- 價格波動率
- 市場信心度

這邊我們重製 2016 年論文 [Khaidem, Luckyson, Snehanshu Saha, and Sudeepa Roy Dey. "Predicting the direction of stock market prices using random forest." arXiv preprint arXiv:1605.00003](https://arxiv.org/abs/1605.00003)

使用 Peter Bakker 提供的 技術指標，包含：

- [相對強弱指數 RSI](https://tw.tradingview.com/scripts/relativestrengthindex/)
- [隨機震盪指標 KD](https://tw.tradingview.com/scripts/stochastic/)
- [累積/派發線 AD](https://tw.tradingview.com/scripts/accumulationdistribution/)
- [真實波動幅度均值 ATR](https://tw.tradingview.com/scripts/volatility/)
- [動量指標 Momentum](https://tw.tradingview.com/scripts/momentum/)
- [資金流量指標 MFI](https://kknews.cc/zh1.tw/invest/b8n5brn.html)
- [變動率指標 ROC](https://tw.tradingview.com/scripts/rateofchange/)
- [能量潮指標 OBV](https://tw.tradingview.com/scripts/onbalancevolume/)
- [順勢指標 CCI](https://tw.tradingview.com/scripts/commoditychannelindex/)
- [簡易波動指標 EOM](https://tw.tradingview.com/scripts/easeofmovement/)
- [三重平滑均線 TRIX](https://tw.tradingview.com/scripts/trix/)
- [旋渦指標 VI](https://tw.tradingview.com/scripts/vortex/)
- [指數平滑異同移動平均線 MACD](https://www.cmoney.tw/learn/course/technicals/topic/750)
- [指數移動平均 EMA](https://ey90223.pixnet.net/blog/post/350258995)

### 加入模組、設定環境常數

In [ ]:
from shutil import copyfile

copyfile(src = "../input/qt-utils/pandas_techinal_indicators.py", dst = "../working/pandas_techinal_indicators.py")

In [ ]:
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
import pandas_techinal_indicators as ta
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split

In [ ]:
%matplotlib inline

In [ ]:
np.random.seed(5)
random.seed(5)

### 設定常數

In [ ]:
# 訓練樣本數比例
SPLIT_RATIO = 0.66

### 加入資料

In [ ]:
spy_prices = pd.read_csv('../input/quantitative-trading/SPY_2018.csv')
del(spy_prices['Date'])
del(spy_prices['Adj Close'])
spy_prices.head()

### 讓資料以時間為權重關聯在一起，但論文作者沒有說明 alpha 值，這邊先用 0.9

In [ ]:
def expPreprocessing(df, alpha=0.9):
    edata = df.ewm(alpha=alpha).mean()    
    return edata

In [ ]:
# Smoothed by Date
sspy = expPreprocessing(spy_prices)
sspy.head()

In [ ]:
def featureExtraction(data):
    for x in [14, 26, 44, 66, 95]:
        data = ta.relative_strength_index(data, n=x)
        data = ta.stochastic_oscillator_d(data, n=x)
        data = ta.accumulation_distribution(data, n=x)
        data = ta.average_true_range(data, n=x)
        data = ta.momentum(data, n=x)
        data = ta.money_flow_index(data, n=x)
        data = ta.rate_of_change(data, n=x)
        data = ta.on_balance_volume(data, n=x)
        data = ta.commodity_channel_index(data, n=x)
        data = ta.ease_of_movement(data, n=x)
        data = ta.trix(data, n=x)
        data = ta.vortex_indicator(data, n=x)
    
    data['ema50'] = data['Close'] / data['Close'].ewm(50).mean()
    data['ema21'] = data['Close'] / data['Close'].ewm(21).mean()
    data['ema14'] = data['Close'] / data['Close'].ewm(14).mean()
    data['ema5'] = data['Close'] / data['Close'].ewm(5).mean()

    # KD 輔助指標
    data = ta.macd(data, n_fast=12, n_slow=26)
    
    del(data['Open'])
    del(data['High'])
    del(data['Low'])
    del(data['Volume'])
    
    return data
   
def computePredictionInt(df, n):
    pred = (df.shift(-n)['Close'] >= df['Close'])
    pred = pred.iloc[:-n]
    return pred.astype(int)

def prepareData(df, horizon):
    data = featureExtraction(df).dropna().iloc[:-horizon]
    data['pred'] = computePredictionInt(data, n=horizon)
    del(data['Close'])
    return data.dropna()


In [ ]:
dataset = prepareData(sspy, 10)
print(f'加入技術指標：{np.array(dataset.columns)}')

### 拆分 訓練資料、測試資料

In [ ]:
features = [x for x in dataset.columns if x not in ['pred']]
x = dataset[features]

y = dataset['pred']

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=SPLIT_RATIO, random_state=5)

print('訓練 > 特徵數量', len(x_train))
print('訓練 > 標籤數量', len(y_train))
print('測試 > 特徵數量', len(x_test))
print('測試 > 標籤數量', len(y_test))

## 建立模型

- n_jobs：-1 代表使用全部核心
- n_estimators：生成多少顆決策樹
- random_state：隨機碼

In [ ]:
Model = RandomForestClassifier(n_jobs=-1, n_estimators=65, random_state=5)
Model.fit(x_train, y_train.values.ravel());

In [ ]:
prediction = Model.predict(x_test)

report = classification_report(y_test, prediction)
confusion = confusion_matrix(y_pred=prediction, y_true=y_test)
print('=== 準確度報告 ===')
print(report)
print('=== 混淆矩陣 ===')
print(confusion)

## 比對 預測結果

In [ ]:
plt.figure(figsize=(20, 7))
plt.plot(np.arange(len(prediction)), prediction, alpha=0.7, label='預測市場漲跌')
plt.plot(np.arange(len(y_test)), y_test, alpha=0.7, label='真實市場漲跌' );
plt.title('測試集中的漲跌預測')
plt.legend();

## 比對 預測機率

In [ ]:
plt.figure(figsize=(20, 7))
probability = Model.predict_proba(x_test)[:, 1]
plt.figure(figsize=(20,7))
plt.plot(np.arange(len(probability)), probability, alpha=0.7, label='預測市場漲跌機率')
plt.plot(np.arange(len(y_test)), y_test, alpha=0.7, label='真實市場漲跌' );
plt.title('測試集中的漲跌機率預測');
plt.legend();

## 結論

論文中使用蘋果股票示範，準確性為 92％，而我們的預測準確度為 90％。原因有很多，可能包括：

- 平滑指數 ewm 的 alpha 不一定是 0.9
- 提取技術指標的日期 [14, 26, 44, 66, 95] 不是(當時)最好的組合
- 目前使用的這些技術指標並不能很好的包含市場情緒

這個方法確實可以預測市場方向，也已經有很多經理人使用此信號作為買賣依據

但請注意！這邊只是示範如何訓練一個模型，並沒有考慮到漲跌的幅度，有時候波動太小時交易獲利會被手續費吃掉要特別注意

另外這個信號是以市場穩定發展為前提，不考慮任何黑天鵝事件(大規模貿易戰、國債凍結)等情況，我們需要多注意市場情緒，並規劃好 止損、止盈 點，剩下的部分嚴格依據信號及策略操作！降低人為因素的干擾